In [63]:
!pip install -U deep-translator
!pip install vaderSentiment

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
# from googletrans import Translator
import re
import nltk
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from deep_translator import GoogleTranslator
from nltk.tokenize import sent_tokenize

In [2]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [133]:
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/zaens/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [124]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Import Data

In [127]:
df = pd.read_csv("youtube_comments.csv", index_col="Unnamed: 0")
df.head()

,author,published_at,comment,like
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0
1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0
3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0


# Translate

In [140]:
def translate_comment(comment):
    if len(comment) < 5000:
        return GoogleTranslator(source='auto', target='en').translate(comment)
    else:
        sentences = sent_tokenize(comment)
        return " ".join([GoogleTranslator(source='auto', target='en').translate(sentence) for sentence in sentences])

In [141]:
df["en_comment"] = df.comment.apply(translate_comment)

In [142]:
# df.to_csv("en_comments.csv")

In [143]:
df = pd.read_csv("en_comments.csv")
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0,Ikn is open to the public...the beauty of the ...
1,1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0,"Does it make sense, starting construction in 2..."
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0,Ikn keep going great. Keep up the spirit of pr...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0,A sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


# Preprocessing

## Drop Missing Value

In [144]:
df.isnull().sum()

Unnamed: 0       0
author           1
published_at     0
comment          0
like             0
en_comment      20
dtype: int64

In [194]:
df.dropna(inplace=True)

In [206]:
df.isnull().sum()

author          0
published_at    0
comment         0
like            0
en_comment      0
dtype: int64

In [147]:
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0,Ikn is open to the public...the beauty of the ...
1,1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0,"Does it make sense, starting construction in 2..."
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0,Ikn keep going great. Keep up the spirit of pr...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0,A sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


## Cleaning

In [148]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [149]:
df.en_comment = df.en_comment.apply(clean_text)
df.comment = df.comment.apply(clean_text)

In [150]:
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...
1,1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


In [202]:
# df.to_csv("clean_comments.csv", index=False)

In [203]:
df = pd.read_csv("clean_comments.csv")
df.head()

,author,published_at,comment,like,en_comment
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


## Labelling 

In [188]:

sentiment = SentimentIntensityAnalyzer()
# text = "dhierp thousand padang food where is the nutrition if you are full you can the one who said it is not nutritious is a national nutritionist in a big city rp thousand how can you get nutritious food boss social assistance oh social assistance"
text = "a sycophantic aggre"
print(labeling(text))
text = "a sycophantic fool"
print(labeling(text))
# x = sentiment.polarity_scores(text)
# x
# labeling(text)
# print(x)
# if x["pos"] > x["neg"]:
#     print("NEGATIF")
# else:
#     print("POSITIF")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
positif
{'neg': 0.592, 'neu': 0.408, 'pos': 0.0, 'compound': -0.4404}
negatif


In [204]:
def labeling(comment):
    sentiment = SentimentIntensityAnalyzer()
    x = sentiment.polarity_scores(comment)
    # print(x)
    if x["compound"] >= 0:
        return "positif"
    else:
        return "negatif"
    # return "positif" if x["compound"] >= 0 else "negatif"

In [207]:
df["label"] = df["en_comment"].apply(labeling)

In [208]:
df.head()

,author,published_at,comment,like,en_comment,label
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...,positif
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...,positif
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...,positif
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool,negatif
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree,positif


In [209]:
df.label.value_counts()

label
positif    2280
negatif     792
Name: count, dtype: int64

In [210]:
# df.to_csv("label_comment.csv")

In [4]:
df = pd.read_csv("label_comment.csv", index_col="Unnamed: 0")
df.head()

,author,published_at,comment,like,en_comment,label
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...,positif
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...,positif
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...,positif
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool,negatif
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree,positif


## Tokenizer

# Dataset Splitting

In [ ]:

train, test = train_test_split(df, test_size=0.2, stratify=,shuffle=False)
train.shape, test.shape

# Dataset & Dataloader

In [ ]:
from torch.utils.data import DataLoader
from jcopdl.utils.dataloader import TimeSeriesDataset

In [ ]:
bs = 32
seq_len = 14
train_set = TimeSeriesDataset(ts_train, "Temp", seq_len=seq_len)
trainloader = DataLoader(train_set, batch_size=bs)


test_set = TimeSeriesDataset(ts_test, "Temp", seq_len=seq_len)
testloader = DataLoader(test_set, batch_size=bs)

# Arsitekstur & Config

In [ ]:
from jcopdl.layers import linear_block

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)
        x = self.fc(x)
        return x, hidden


In [ ]:
config = set_config({
    "input_size" : train_set.n_feature,
    "output_size" : 1,
    "hidden_size" : 64,
    "num_layers" : 3,
    "dropout" : 0,
    "seq_len" : train_set.n_seq,
})

# Training Preparation

In [ ]:
model = GRU(config.input_size, config.output_size, config.hidden_size, config.num_layers, config.dropout).to(device)
criterion = nn.MSELoss(reduction="mean")
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config=config, outdir="model/lstm")

# Training

In [ ]:
from tqdm.auto import tqdm
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output, hidden = model(feature, None) # feedforward
        loss = criterion(output, target)
        if mode == "train":
            loss.backward() # backpropragation
            optimizer.step() # update weight
            optimizer.zero_grad()
        cost += loss.item() * feature.shape[0]
    cost = cost / len(dataset)
    return cost